In [1]:

import json
import os
import polars as pl
from polars import col as c
import polars.selectors as cs
import re
from data_federation.input_model import SmallflexInputSchema
import owncloud
import streamlit as st
from datetime import date
import plotly.graph_objs as go
from data_federation.parser.market_price import parse_market_price
from data_federation.parser.discharge_flow import parse_discharge_flow
from data_federation.parser.weather import parse_weather
from data_federation.parser.aegina_hydro_power_plant import parse_aegina_hydro_power_plant
from data_federation.parser.merezenbach_hydro_power_plant import parse_merezenbach_hydro_power_plant
from data_federation.parser.morel_hydro_power_plant import parse_morel_hydro_power_plant
from data_federation.parser.aegina_wind_power_plant import parse_aegina_wind_power_plant
from data_federation.parser.parser_pipeline import input_pipeline
from utility.general_function import dictionary_key_filtering, scan_switch_directory, modify_string, generate_uuid, pl_to_dict
from utility.polars_operation import modify_string_col, generate_uuid_col

from data_federation.parser.hydro_power_plant import get_hydro_power_plant_data
from config import settings
import plotly.express as px

from itertools import product

os.chdir(os.getcwd().replace("/src", ""))




In [2]:

input_file_names: dict[str, str] = json.load(open(settings.INPUT_FILE_NAMES))
output_file_names: dict[str, str] = json.load(open(settings.OUTPUT_FILE_NAMES))
market_price_metadata: dict = json.load(open(input_file_names["market_price_metadata"]))


In [48]:
sorted(market_price_measurement["market"].unique().to_list())

['DA', 'FCR-cap', 'IDA', 'aFRR-act', 'aFRR-cap', 'mFRR-act', 'mFRR-cap']

In [14]:
output_file_names: dict[str, str] = json.load(open(settings.OUTPUT_FILE_NAMES))
small_flex_input_schema: SmallflexInputSchema = SmallflexInputSchema()\
    .duckdb_to_schema(file_path=output_file_names["duckdb_input"])
    
    


# kwargs["small_flex_input_schema"] 

Read and validate tables from small_flex_input_data.db file: 100%|████████████████████████████████████████████████████| 12/12 [00:04<00:00,  2.69it/s]


In [8]:


# date_index = 
date_mapping = pl_to_dict(
    pl.DataFrame(pl.datetime_range(date(2022, 1, 1), date(2023, 1, 1), "4h", eager=True, closed="left").alias("timestamp"))\
    .with_row_index("index")\
    .filter(~((c("timestamp").dt.month() == 2) & (c("timestamp").dt.day() == 29)))\
    .with_columns(
        c("timestamp").dt.to_string(format="%m-%d %H:%M").alias("date_str"),
    )["date_str", "index"])

In [16]:

actual_market = market_price_measurement.filter(c("market") == "mFRR-cap")

In [24]:
year_list = sorted(data_country["year"].unique().to_list())
direction_list = data_country["direction"].unique()

In [28]:
for i, year in enumerate(product(year_list, direction_list)):
    print(i, year)
    print(i % len(direction_list))

0 (2019, 'pos')
0
1 (2020, 'pos')
0
2 (2021, 'pos')
0
3 (2022, 'pos')
0
4 (2023, 'pos')
0


In [45]:
from plotly.subplots import make_subplots
# date_index = 
colors = px.colors.qualitative.Plotly

market = "DA"

timestep = "1h" if market in ["DA", "IDA"] else "4h"

date_mapping = pl_to_dict(
    pl.DataFrame(pl.datetime_range(
        start=date(2022, 1, 1), end=date(2023, 1, 1), interval=timestep,
        eager=True, closed="left").alias("timestamp")
    ).with_row_index("index")\
    .filter(~((c("timestamp").dt.month() == 2) & (c("timestamp").dt.day() == 29)))\
    .with_columns(
        c("timestamp").dt.to_string(format="%m-%d %H:%M").alias("date_str"),
    )["date_str", "index"])

data = market_price_measurement.filter(c("market") == "mFRR-cap").with_columns(
    c("date_str").replace_strict(date_mapping, default=None).alias("index"),
)
tabs_lv_2 = st.tabs(data["country"].unique().to_list())
for tab_idx, country in enumerate(data["country"].unique().to_list()):
    trace_list: list = []
    data_by_country = data.filter(c("country")==country)
    year = sorted(data_by_country["year"].unique().to_list())
    direction_list = actual_market["direction"].unique()
    fig = make_subplots(rows=len(direction_list), cols = 1, subplot_titles=direction_list, shared_xaxes=True)
    for i, values in enumerate(product(year_list, direction_list)):
        year = values[0]
        direction = values[1]
        fig_index = i % len(direction_list) + 1
        
        color_index= i//2
        plot_data = data_by_country.filter(c("year") == year).filter(c("direction") == direction).sort("index")
        fig.add_trace(
            go.Scatter(
                x=plot_data["index"].to_list(), y=plot_data["avg"].to_list(),
                mode='lines', name=str(year), legendgroup=str(year),  line=dict(color=colors[color_index]),
                showlegend=fig_index==1),
            row=fig_index, col=1
            )

    fig.update_layout(xaxis=dict(nticks=11))

    with tabs_lv_2[tab_idx]:
            st.plotly_chart(fig, theme="streamlit", use_container_width=True)

NameError: name 'timestep' is not defined

In [42]:
fig.show

In [13]:
energy = ".cache/input/alpiq_input_data/swissgrid/energy/2020-03-TRE-Ergebnis.csv"

data = pl.read_csv(energy, separator=";", encoding='iso-8859-1', null_values=["*", "N/A"])


data

Ausschreibung,Von,Bis,Produkt,Angebotene Menge,Einheit,Abgerufene Menge,Einheit_duplicated_0,Preis,Einheit_duplicated_1,Status
str,str,str,str,i64,str,i64,str,f64,str,str
"""TRE_20_03_01""","""00:00""","""01:00""","""TREnergie+_s""",5,"""MW""",0,"""MW""",499.0,"""EUR/MWh""","""verfügbar"""
"""TRE_20_03_01""","""00:00""","""01:00""","""TREnergie+_s""",5,"""MW""",0,"""MW""",272.0,"""EUR/MWh""","""verfügbar"""
"""TRE_20_03_01""","""00:00""","""01:00""","""TREnergie+_s""",5,"""MW""",0,"""MW""",92.0,"""EUR/MWh""","""verfügbar"""
"""TRE_20_03_01""","""00:00""","""01:00""","""TREnergie+_s""",5,"""MW""",0,"""MW""",374.0,"""EUR/MWh""","""verfügbar"""
"""TRE_20_03_01""","""00:00""","""01:00""","""TREnergie+_s""",5,"""MW""",0,"""MW""",81.0,"""EUR/MWh""","""verfügbar"""
…,…,…,…,…,…,…,…,…,…,…
"""TRE_20_03_31""","""23:00""","""00:00""","""TREnergie-_s""",25,"""MW""",0,"""MW""",10.0,"""EUR/MWh""","""verfügbar"""
"""TRE_20_03_31""","""23:00""","""00:00""","""TREnergie-_s""",25,"""MW""",0,"""MW""",6.0,"""EUR/MWh""","""verfügbar"""
"""TRE_20_03_31""","""23:00""","""00:00""","""TREnergie-_s""",25,"""MW""",0,"""MW""",14.0,"""EUR/MWh""","""verfügbar"""


In [42]:
market_price_measurement

timestamp,market,direction,min,avg,max,country,source,unit
"datetime[μs, UTC]",str,str,f64,f64,f64,str,str,str
2020-11-21 20:00:00 UTC,"""mFRR-cap""","""pos""",0.0,0.144331,0.24,"""CH""","""swissgrid""","""EUR/MWh"""
2020-12-13 20:00:00 UTC,"""mFRR-cap""","""pos""",0.0,0.183571,0.48,"""CH""","""swissgrid""","""EUR/MWh"""
2020-01-22 08:00:00 UTC,"""mFRR-cap""","""pos""",0.45,0.474096,0.5,"""CH""","""swissgrid""","""EUR/MWh"""
2020-08-25 00:00:00 UTC,"""mFRR-cap""","""pos""",0.0,0.002,0.01,"""CH""","""swissgrid""","""EUR/MWh"""
2020-11-14 20:00:00 UTC,"""FCR-cap""","""sym""",11.72,12.339788,33.08,"""CH""","""swissgrid""","""EUR/MWh"""
…,…,…,…,…,…,…,…,…
2015-01-29 00:00:00 UTC,"""mFRR-cap""","""neg""",0.98,0.998824,1.0,"""CH""","""swissgrid""","""EUR/MWh"""
2015-03-31 08:00:00 UTC,"""mFRR-cap""","""neg""",0.72,3.394839,4.75,"""CH""","""swissgrid""","""EUR/MWh"""
2015-03-27 08:00:00 UTC,"""mFRR-cap""","""pos""",6.25,10.104925,12.25,"""CH""","""swissgrid""","""EUR/MWh"""


In [15]:
data

market,y,m,d,time,Angebotenes Volumen,Einheit,quantity,Einheit_duplicated_0,Leistungspreis,Einheit_duplicated_1,Kosten,Einheit_duplicated_2,price,Einheit_duplicated_3,Land,Angebotspreis,Einheit_duplicated_4,Teilbarkeit
str,str,str,str,str,i64,str,i64,str,f64,str,f64,str,f64,str,str,f64,str,str
"""PRL""","""20""","""01""","""01""","""Auction""",16,"""MW""",16,"""MW""",50.0,"""EUR/MW""",800.0,"""EUR""",6.21,"""EUR/MWh*""","""CH""",2.08,"""EUR/MWh*""","""Ja"""
"""PRL""","""20""","""01""","""01""","""Auction""",1,"""MW""",1,"""MW""",80.0,"""EUR/MW""",80.0,"""EUR""",6.21,"""EUR/MWh*""","""CH""",3.33,"""EUR/MWh*""","""Ja"""
"""PRL""","""20""","""01""","""01""","""Auction""",1,"""MW""",1,"""MW""",80.0,"""EUR/MW""",80.0,"""EUR""",6.21,"""EUR/MWh*""","""CH""",3.33,"""EUR/MWh*""","""Ja"""
"""PRL""","""20""","""01""","""01""","""Auction""",1,"""MW""",1,"""MW""",80.0,"""EUR/MW""",80.0,"""EUR""",6.21,"""EUR/MWh*""","""CH""",3.33,"""EUR/MWh*""","""Ja"""
"""PRL""","""20""","""01""","""01""","""Auction""",1,"""MW""",1,"""MW""",120.0,"""EUR/MW""",120.0,"""EUR""",6.21,"""EUR/MWh*""","""CH""",5.0,"""EUR/MWh*""","""Ja"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""TRL+""","""20""","""12""","""31""","""20:00""",10,"""MW""",10,"""MW""",0.35,"""CHF/MW""",3.5,"""CHF""",0.09,"""CHF/MWh*""","""CH""",0.09,"""CHF/MWh*""",null
"""TRL+""","""20""","""12""","""31""","""20:00""",10,"""MW""",10,"""MW""",0.96,"""CHF/MW""",9.6,"""CHF""",0.24,"""CHF/MWh*""","""CH""",0.24,"""CHF/MWh*""",null
"""TRL+""","""20""","""12""","""31""","""20:00""",10,"""MW""",10,"""MW""",1.8,"""CHF/MW""",18.0,"""CHF""",0.45,"""CHF/MWh*""","""CH""",0.45,"""CHF/MWh*""",null


In [ ]:
        .with_columns(
            pl.concat_str(["y", "m", "d", "time"], separator="-")
            .str.to_datetime("%y-%m-%d-%H:%M", time_zone="UTC").alias("timestamp"),
        ).group_by(["timestamp", "market"]).agg(
            c("price").min().alias("min"),
            ((c("price") *c("quantity")).sum()/c("quantity").sum()).alias("avg"),
            c("price").max().alias("max"),
        ).with_columns(
            c("market").replace_strict(market_price_metadata[name]["market_mapping"])
        ).with_columns(
            pl.lit(value).alias(name) for name, value in market_price_metadata[name]["data"].items()
        ).unnest("market")

# data = data.rename(market_price_metadata[name]["col_mapping"]).filter(pl.col("quantity") > 0)

In [7]:
data

Ausschreibung,Beschreibung,Angebotenes Volumen,Einheit,Zugesprochenes Volumen,Einheit_duplicated_0,Leistungspreis,Einheit_duplicated_1,Kosten,Einheit_duplicated_2,Preis,Einheit_duplicated_3,Land,Angebotspreis,Einheit_duplicated_4,Teilbarkeit
str,str,i64,str,i64,str,f64,str,f64,str,f64,str,str,f64,str,str
"""PRL_20_01_01""","""Primary control Auction""",16,"""MW""",16,"""MW""",50.0,"""EUR/MW""",800.0,"""EUR""",6.21,"""EUR/MWh*""","""CH""",2.08,"""EUR/MWh*""","""Ja"""
"""PRL_20_01_01""","""Primary control Auction""",1,"""MW""",1,"""MW""",80.0,"""EUR/MW""",80.0,"""EUR""",6.21,"""EUR/MWh*""","""CH""",3.33,"""EUR/MWh*""","""Ja"""
"""PRL_20_01_01""","""Primary control Auction""",1,"""MW""",1,"""MW""",80.0,"""EUR/MW""",80.0,"""EUR""",6.21,"""EUR/MWh*""","""CH""",3.33,"""EUR/MWh*""","""Ja"""
"""PRL_20_01_01""","""Primary control Auction""",1,"""MW""",1,"""MW""",80.0,"""EUR/MW""",80.0,"""EUR""",6.21,"""EUR/MWh*""","""CH""",3.33,"""EUR/MWh*""","""Ja"""
"""PRL_20_01_01""","""Primary control Auction""",1,"""MW""",1,"""MW""",120.0,"""EUR/MW""",120.0,"""EUR""",6.21,"""EUR/MWh*""","""CH""",5.0,"""EUR/MWh*""","""Ja"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""TRL+_20_12_31""","""Tertiary Power UP 20:00 bis 24…",10,"""MW""",0,"""MW""",14.0,"""CHF/MW""",0.0,"""CHF""",3.5,"""CHF/MWh*""","""CH""",3.5,"""CHF/MWh*""",null
"""TRL+_20_12_31""","""Tertiary Power UP 20:00 bis 24…",10,"""MW""",0,"""MW""",20.0,"""CHF/MW""",0.0,"""CHF""",5.0,"""CHF/MWh*""","""CH""",5.0,"""CHF/MWh*""",null
"""TRL+_20_12_31""","""Tertiary Power UP 20:00 bis 24…",10,"""MW""",10,"""MW""",2.16,"""CHF/MW""",21.6,"""CHF""",0.54,"""CHF/MWh*""","""CH""",0.54,"""CHF/MWh*""",null


In [ ]:
head_table

In [ ]:
data_pl

In [ ]:
data_pl.select(
    pl.struct(cs.contains("Pe_") != 0).alias("resource_state")
)

In [ ]:
precipitation = data.unpivot(
        index=["timestamp"], variable_name="metadata", value_name="value", on=cs.starts_with("prec_")
    ).with_columns(
    ("gries_" + c("metadata").str.slice(-3, 1)).alias("sub_basin"),
    ((c("metadata").str.slice(-2) + "00").cast(pl.Int32)-50).alias("start_height")
)


In [ ]:
precipitation

In [ ]:
data.columns

In [ ]:
small_flex_input_schema

In [ ]:
discharge_flow

In [ ]:
data_str = str(ro.r['df'])
data_str = (list(map(lambda x: modify_string(x, format_str={r"\s+": " "}).split(" "), data_str.split("\n"))))


index = [i for i in range(len(data_str)) if data_str[i][0] == ""]
for i in range(len(index)-2):
    if i == 0:
        columns = ["index", "date"] + data_str[index[i]][1:]
        data: pl.DataFrame = pl.DataFrame(data_str[index[i] + 1:index[i+1]], schema=columns, orient="row")
    else:
        columns = ["index"] + data_str[index[i]][1:]
        data = data.join(
            pl.DataFrame(data_str[index[i] + 1:index[i+1]], schema=columns, orient="row"), on="index", how="left")


In [ ]:

data = data.with_columns(
    pl.concat_str(["date", "time"], separator=" ").str.to_datetime("%Y-%m-%d %H:%M:%S", time_zone="UTC").alias("timestamp")
)

wind_data = data.unpivot(on=cs.contains("wind"), index="timestamp", value_name="wind_speed", variable_name="height")\
    .with_columns(
        c("height").str.replace("wind_X", ""),
        c("wind_speed").cast(pl.Float64),
    )

irradiance_data = data.unpivot(on=cs.contains("glob"), index="timestamp", value_name="irradiance", variable_name="height")\
    .with_columns(
        c("height").str.replace("glob__X", ""),
        c("irradiance").cast(pl.Float64),
    )

irradiance_data

In [ ]:
data

In [ ]:
list(map(lambda x: x.split(" "), data_str.split("\n")))


In [ ]:
data = pl.from_pandas(pandas2ri.rpy2py())
print(data.head())

In [ ]:
ro.r['df']

In [ ]:
df_temp2 =data.filter(pl.col("metadata").str.contains("KW"))\
    .with_columns(
        c("metadata").str.replace("KW", "").
        str.split_exact("_", 4).struct.rename_fields(["market", "y", "w"]).alias("metadata"),
    ).unnest("metadata").with_columns(
        (c("y").str.to_datetime("%y", time_zone="UTC") + 
        ((pl.col("w").cast(pl.Int32) - 1)*7*24*60*60*1e3).cast(pl.Duration(time_unit="ms"))).alias("timestamp"),
    )
df_temp2.head(1)


In [ ]:
    .with_columns(
        pl.col("Date").str.split("_").map_elements(lambda x: x[0]).alias("market"),
        pl.col("Date").str.split("_").map_elements(lambda x: x[1]).alias("y").cast(pl.Int64),
        pl.col("Date").str.split("KW").map_elements(lambda x: x[-1]).alias("week").str.split("_")
        .map_elements(lambda x: x[0]).cast(pl.Int64)]).with_columns(
            ((pl.col("week") - 1)*7*24*60*60*1e6).cast(pl.Duration(time_unit="us")),
                (pl.col("y") + 2000).cast(pl.Utf8).str.strptime(pl.Datetime, "%Y")
            )


In [ ]:
name = "rte_ene"
market_price: pl.DataFrame = pl.read_csv(
    market_price_metadata[name]["file"], separator=";", null_values=["*"])

market_price.columns